# Cancer Diagnosis Prediction 

## Feature Selection

Approach II: 
1. Train boosting models - Light GBM, CatBoost, XGBoost with base dataset to which Lasso Regression has been applied as well as well as highly correlated features has been removed. Plus one deep learning model - TabNet. 
2. Train models - Light GBM, CatBoost, XGBoost and TabNet with using important features from Random Forest. 
2. Devide features by locations (`Region`, `Division`, `patient_state`) and apply Lasso Regression to each location separately to select important features
3. Apply a boosting model to each location. 

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
train_base = pd.read_csv('target_encoded_train.csv')
test_base = pd.read_csv('target_encoded_test.csv')

In [4]:
train_base

,patient_id,patient_race,payer_type,patient_state,patient_zip3,patient_age,breast_cancer_diagnosis_code,metastatic_cancer_diagnosis_code,Region,Division,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
0,475714,0.605515,0.610440,0.645148,924,84,0.781037,0.593023,0.653121,0.643909,31437.75000,1189.562500,30.642857,16.014286,15.542857,17.614286,14.014286,11.614286,11.557143,7.571429,4.000000,2.100000,49.857143,50.142857,36.571429,11.885714,47.114286,4.442857,3.928571,52.228571,52996.28571,3.142857,4.000000,6.157143,5.142857,6.271429,10.142857,13.300000,20.000000,12.742857,11.571429,7.528571,19.100000,24563.57143,44.585714,8674.500000,2.646343e+05,1165.000000,37.442857,33.257143,29.200000,25.914286,8.357143,3.257143,11.614286,39.557143,61.528571,8.471429,13.428571,0.000000,44.100000,13.100000,5.100000,1.485714,0.342857,27.114286,8.757143,66.685714,12.871429,22.542857,10.100000,27.814286,11.200000,3.500000,52.237210,8.650555,18.606528,1
1,349367,0.659142,0.599005,0.645148,928,62,0.783231,0.669078,0.653121,0.643909,39121.87879,2295.939394,38.200000,11.878788,13.354545,14.230303,13.418182,13.333333,14.060606,10.248485,5.951515,3.503030,49.893939,50.106061,50.245455,9.827273,35.290909,4.651515,3.622727,61.736364,102741.63640,2.327273,1.536364,2.648485,2.178788,2.409091,5.163636,7.972727,13.936364,12.469697,19.760606,29.596970,49.357576,41287.27273,61.463636,11725.666670,6.776885e+05,2003.125000,34.753125,14.230303,19.987879,29.796970,23.739394,12.245455,35.984848,47.918182,65.230303,5.103030,15.224242,0.027273,54.030303,2.527273,20.827273,0.587879,0.300000,11.645455,10.081818,37.948485,8.957576,10.109091,8.057576,30.606061,7.018182,4.103030,42.301121,8.487175,20.113179,1
2,138632,0.659142,0.599005,0.599649,760,43,0.760766,0.669078,0.603948,0.609875,21996.68333,626.236667,37.906667,13.028333,14.463333,12.531667,13.545000,12.860000,12.770000,11.426667,6.565000,2.811667,50.123333,49.876667,55.753333,12.330000,27.195000,4.710000,3.260667,55.801667,85984.74138,2.483333,1.305000,2.716667,2.938333,2.766667,6.763333,12.061667,15.835000,13.560000,20.875000,18.680000,39.555000,40399.03333,72.745000,7786.583333,2.377131e+05,1235.907407,29.358491,10.811667,27.038333,32.368333,19.678333,10.115000,29.793333,37.308475,66.428333,4.560000,13.722034,3.650847,75.820000,9.231667,3.618333,0.463333,0.146667,3.816667,6.898333,19.370000,11.253333,9.663333,3.356667,31.394915,15.066667,7.446667,40.108207,7.642753,14.839351,1
3,617843,0.659142,0.599005,0.645148,926,45,0.774744,0.669078,0.653121,0.643909,32795.32558,1896.220930,42.871429,10.071429,12.135714,12.538095,12.464286,12.650000,14.847619,12.280952,8.216667,4.759524,49.066667,50.933333,52.604762,11.623810,31.142857,4.623810,3.098095,54.564286,120533.83330,3.435714,1.273810,2.180952,2.211905,2.100000,4.380952,5.885714,10.897619,10.721429,18.850000,38.057143,56.907143,55336.28571,59.221429,12171.302330,1.012474e+06,2354.738095,32.030952,5.835714,12.145238,26.269048,33.285714,22.459524,55.745238,48.938095,64.430952,5.264286,18.50238

In [5]:
patient_id = test_base['patient_id']

train_base.drop(columns=['patient_id'], axis=1, inplace=True)
test_base.drop(columns=['patient_id'], axis=1, inplace=True)

In [6]:
train_base


,patient_race,payer_type,patient_state,patient_zip3,patient_age,breast_cancer_diagnosis_code,metastatic_cancer_diagnosis_code,Region,Division,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
0,0.605515,0.610440,0.645148,924,84,0.781037,0.593023,0.653121,0.643909,31437.75000,1189.562500,30.642857,16.014286,15.542857,17.614286,14.014286,11.614286,11.557143,7.571429,4.000000,2.100000,49.857143,50.142857,36.571429,11.885714,47.114286,4.442857,3.928571,52.228571,52996.28571,3.142857,4.000000,6.157143,5.142857,6.271429,10.142857,13.300000,20.000000,12.742857,11.571429,7.528571,19.100000,24563.57143,44.585714,8674.500000,2.646343e+05,1165.000000,37.442857,33.257143,29.200000,25.914286,8.357143,3.257143,11.614286,39.557143,61.528571,8.471429,13.428571,0.000000,44.100000,13.100000,5.100000,1.485714,0.342857,27.114286,8.757143,66.685714,12.871429,22.542857,10.100000,27.814286,11.200000,3.500000,52.237210,8.650555,18.606528,1
1,0.659142,0.599005,0.645148,928,62,0.783231,0.669078,0.653121,0.643909,39121.87879,2295.939394,38.200000,11.878788,13.354545,14.230303,13.418182,13.333333,14.060606,10.248485,5.951515,3.503030,49.893939,50.106061,50.245455,9.827273,35.290909,4.651515,3.622727,61.736364,102741.63640,2.327273,1.536364,2.648485,2.178788,2.409091,5.163636,7.972727,13.936364,12.469697,19.760606,29.596970,49.357576,41287.27273,61.463636,11725.666670,6.776885e+05,2003.125000,34.753125,14.230303,19.987879,29.796970,23.739394,12.245455,35.984848,47.918182,65.230303,5.103030,15.224242,0.027273,54.030303,2.527273,20.827273,0.587879,0.300000,11.645455,10.081818,37.948485,8.957576,10.109091,8.057576,30.606061,7.018182,4.103030,42.301121,8.487175,20.113179,1
2,0.659142,0.599005,0.599649,760,43,0.760766,0.669078,0.603948,0.609875,21996.68333,626.236667,37.906667,13.028333,14.463333,12.531667,13.545000,12.860000,12.770000,11.426667,6.565000,2.811667,50.123333,49.876667,55.753333,12.330000,27.195000,4.710000,3.260667,55.801667,85984.74138,2.483333,1.305000,2.716667,2.938333,2.766667,6.763333,12.061667,15.835000,13.560000,20.875000,18.680000,39.555000,40399.03333,72.745000,7786.583333,2.377131e+05,1235.907407,29.358491,10.811667,27.038333,32.368333,19.678333,10.115000,29.793333,37.308475,66.428333,4.560000,13.722034,3.650847,75.820000,9.231667,3.618333,0.463333,0.146667,3.816667,6.898333,19.370000,11.253333,9.663333,3.356667,31.394915,15.066667,7.446667,40.108207,7.642753,14.839351,1
3,0.659142,0.599005,0.645148,926,45,0.774744,0.669078,0.653121,0.643909,32795.32558,1896.220930,42.871429,10.071429,12.135714,12.538095,12.464286,12.650000,14.847619,12.280952,8.216667,4.759524,49.066667,50.933333,52.604762,11.623810,31.142857,4.623810,3.098095,54.564286,120533.83330,3.435714,1.273810,2.180952,2.211905,2.100000,4.380952,5.885714,10.897619,10.721429,18.850000,38.057143,56.907143,55336.28571,59.221429,12171.302330,1.012474e+06,2354.738095,32.030952,5.835714,12.145238,26.269048,33.285714,22.459524,55.745238,48.938095,64.430952,5.264286,18.502381,0.052381,65.014286,1.438095,18.845238

In [7]:
# Group by 'patient_state' and count the occurrences of each value in 'DiagPeriodL90D'
agg_results = train_base.groupby('Division')['DiagPeriodL90D'].value_counts()
agg_results

Division  DiagPeriodL90D
0.578629  1                  287
          0                  209
0.590361  1                  931
          0                  646
0.605999  1                 1192
          0                  775
0.609875  1                  877
          0                  561
0.623678  1                 1828
          0                 1103
0.643909  1                 1792
          0                  991
0.666209  1                  485
          0                  243
0.679226  1                  667
          0                  315
Name: count, dtype: int64

In [8]:
# Reset the index to turn the multi-index series into a DataFrame
agg_results_df = agg_results.reset_index(name='count')
agg_results_df[agg_results_df['count'] > 40]

,Division,DiagPeriodL90D,count
0,0.578629,1,287
1,0.578629,0,209
2,0.590361,1,931
3,0.590361,0,646
4,0.605999,1,1192
5,0.605999,0,775
6,0.609875,1,877
7,0.609875,0,561
8,0.623678,1,1828
9,0.623678,0,1103


In [9]:
train_base.DiagPeriodL90D.value_counts()

DiagPeriodL90D
1    8059
0    4843
Name: count, dtype: int64

In [10]:
target = train_base['DiagPeriodL90D']
train_base.drop(columns=target.name, axis=1, inplace=True) 

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
import numpy as np

def apply_lasso(df_slice, target, alpha=0.1,):
    # Splitting data into features (X) and target variable (y)
    y = target
    X = df_slice

    # Filter features based on correlation
    corr_matrix = X.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]
    X_filtered = X.drop(columns=to_drop)

    # Standardize the Features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Fit LASSO regression
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_scaled, y)


    # Extract important features
    lasso_coefficients = lasso_model.coef_
    important_features = X.columns[np.abs(lasso_coefficients) > 0]

    return list(important_features)

In [12]:
from boosting_models import apply_catboost, apply_lightgbm, apply_xgboost
from tabnet import apply_tabnet
from tqdm import tqdm

In [13]:
important_features = apply_lasso(train_base, target, alpha=0.001)
important_features

['payer_type',
 'patient_state',
 'patient_zip3',
 'patient_age',
 'breast_cancer_diagnosis_code',
 'metastatic_cancer_diagnosis_code',
 'age_under_10',
 'age_40s',
 'age_50s',
 'age_over_80',
 'family_size',
 'family_dual_income',
 'income_household_under_5',
 'income_household_20_to_25',
 'income_household_25_to_35',
 'income_household_35_to_50',
 'income_household_50_to_75',
 'income_household_75_to_100',
 'income_individual_median',
 'housing_units',
 'home_value',
 'rent_burden',
 'education_less_highschool',
 'education_some_college',
 'education_stem_degree',
 'unemployment_rate',
 'self_employed',
 'race_black',
 'race_asian',
 'race_native',
 'race_other',
 'race_multiple',
 'limited_english',
 'commute_time',
 'health_uninsured',
 'veteran',
 'Ozone',
 'PM25',
 'N02']

In [14]:
best_catboost_model, best_cat_params, best_cat_score = apply_catboost(train_base[important_features], target, cv=5)
print("Best Parameters for model:", best_cat_params)
print("Best ROC for model:", best_cat_score)

Best Parameters for model: {'depth': 4, 'iterations': 300, 'learning_rate': 0.05}
Best ROC for model: 0.8053374536931989


**Results of CatBoost**

With lasso, alpha - 0.006:
- Best Parameters for model: {'depth': 4, 'iterations': 300, 'learning_rate': 0.05}
- Best ROC for model: 0.8081640047109044

With lasso, alpha - 0.01:
- Best Parameters for model: {'depth': 4, 'iterations': 200, 'learning_rate': 0.05}
- Best ROC for model: 0.8019676819045394

With target encoded cats:
CatBoost shows only 0.55 ROC on test data. 
- Best Parameters for model: {'depth': 4, 'iterations': 200, 'learning_rate': 0.05}
- Best ROC for model: 0.8095736565396061

In [15]:
best_gbm_model, best_gbm_params, best_gmb_score = apply_lightgbm(train_base[important_features], target, cv=5)
print("Best Parameters for model:", best_gbm_params)
print("Best ROC for model:", best_gmb_score)

Best Parameters for model: {'learning_rate': 0.05, 'n_estimators': 50, 'num_leaves': 31}
Best ROC for model: 0.7993186123815711


**Results of light GBM**

With lasso, alpha - 0.006:
- Best Parameters for model: {'learning_rate': 0.05, 'n_estimators': 50, 'num_leaves': 31}
- Best ROC for base model: 0.8014304888703222

With lasso, alpha - 0.01:
- Best Parameters for model: {'learning_rate': 0.01, 'n_estimators': 150, 'num_leaves': 31}
- Best ROC for base model: 0.7959317174144424

With features selected by Random Forest:
- Best Parameters for model: {'learning_rate': 0.05, 'n_estimators': 50, 'num_leaves': 31}
- Best ROC for base model: 0.7941765738763503

With target encoded cats (**best result on test 0,761**):
- Best Parameters for model: {'learning_rate': 0.05, 'n_estimators': 50, 'num_leaves': 31}
- Best ROC for model: 0.8026098173768395

In [16]:
best_xgb_model, best_xgb_params, best_xgb_score = apply_xgboost(train_base[important_features], target, cv=5)
print("Best Parameters for model:", best_xgb_params)
print("Best ROC for model:", best_xgb_score)

Best Parameters for model: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50}
Best ROC for model: 0.8038924317898439


**Results of XGBoost**

With lasso, alpha - 0.006:
- Best Parameters for model: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 150}
- Best ROC for base model: 0.8026676590394187

With lasso, alpha - 0.01:
- Best Parameters for model: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 150}
- Best ROC for base model: 0.8001561991412572

With target encoded cats:
- Best Parameters for model: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 150}
- Best ROC for model: 0.8072741202015724

In [17]:
best_tabnet_model, best_tabnet_params, best_tabnet_score = apply_tabnet(train_base[important_features], target, cv=5)
print("Best Parameters for model:", best_tabnet_params)
print("Best ROC for model:", best_tabnet_score)

C:\Users\Julia\AppData\Roaming\Python\Python310\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.81129 | val_0_auc: 0.64575 |  0:00:01s
epoch 1  | loss: 0.57851 | val_0_auc: 0.74931 |  0:00:03s
epoch 2  | loss: 0.50408 | val_0_auc: 0.76676 |  0:00:04s
epoch 3  | loss: 0.48668 | val_0_auc: 0.77589 |  0:00:06s
epoch 4  | loss: 0.48119 | val_0_auc: 0.78171 |  0:00:07s
epoch 5  | loss: 0.47489 | val_0_auc: 0.78534 |  0:00:09s
epoch 6  | loss: 0.47703 | val_0_auc: 0.78611 |  0:00:10s
epoch 7  | loss: 0.47395 | val_0_auc: 0.7878  |  0:00:12s
epoch 8  | loss: 0.47227 | val_0_auc: 0.79091 |  0:00:13s
epoch 9  | loss: 0.47318 | val_0_auc: 0.79371 |  0:00:15s
epoch 10 | loss: 0.47096 | val_0_auc: 0.79684 |  0:00:17s
epoch 11 | loss: 0.4687  | val_0_auc: 0.79651 |  0:00:18s
epoch 12 | loss: 0.4705  | val_0_auc: 0.79979 |  0:00:20s
epoch 13 | loss: 0.47148 | val_0_auc: 0.79884 |  0:00:22s
epoch 14 | loss: 0.47049 | val_0_auc: 0.8014  |  0:00:23s
epoch 15 | loss: 0.47053 | val_0_auc: 0.80012 |  0:00:25s
epoch 16 | loss: 0.47009 | val_0_auc: 0.80127 |  0:00:27s
epoch 17 | los

C:\Users\Julia\AppData\Roaming\Python\Python310\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Best Parameters for model: None
Best ROC for model: 0.8412950617627785


**Results of TabNet**

With lasso, alpha - 0.006:
- Best ROC for model: 0.8329930714111653

With lasso, alpha - 0.01:
- Best ROC for model: 0.8136765948486919 

With features selected by Random Forest:
- Best ROC for model: 0.8103170282700086

With target encoded cats:
- Best ROC for model: 0.830307913681304


In [18]:
def test_and_pack(test, important_features, best_model, name):
    scaler = StandardScaler()
    scaled = scaler.fit_transform(test[important_features])

    pred_y = best_model.predict(scaled)

    preds = pd.DataFrame(pred_y, columns=['DiagPeriodL90D'])
    preds.insert(0, 'patient_id', patient_id)

    preds.to_csv(f'{name}_result.csv', index=False)

In [19]:
test_and_pack(test_base, important_features, best_tabnet_model, 'tabnet_target_encoded')
test_and_pack(test_base, important_features, best_catboost_model, 'cat_target_encoded')
test_and_pack(test_base, important_features, best_gbm_model, 'gbm_target_encoded')
test_and_pack(test_base, important_features, best_xgb_model, 'xgb_target_encoded')

## Train Models on Locations

In [50]:
train = pd.read_csv('target_encoded_train.csv')
test = pd.read_csv('target_encoded_test.csv')

In [51]:
# train = pd.get_dummies(train, columns=[
#     'patient_race', 
#     'payer_type',
#     'breast_cancer_diagnosis_code',
#     'metastatic_cancer_diagnosis_code'
#     ], dtype=float)
# test = pd.get_dummies(test, columns=[
#     'patient_race', 
#     'payer_type',
#     'breast_cancer_diagnosis_code',
#     'metastatic_cancer_diagnosis_code'
#     ], dtype=float)

In [52]:
def group_by_location(train, test, location_type):
    
    drop_columns = {
        'patient_state': ['Division', 'Region'],
        'Region': ['Division', 'patient_state'],
        'Division': ['patient_state', 'Region']
    }

    columns_to_drop = drop_columns.get(location_type, [])

    train = train.drop(columns_to_drop, axis=1)
    test = test.drop(columns_to_drop, axis=1)


    # Get unique locations
    unique_locations = train[location_type].unique()

    # Create a dictionary to store DataFrames for each location
    locations_dfs = {}
    target_dfs = {}

    # Iterate over unique locations
    for location in unique_locations:
        # Filter DataFrame for the current state
        location_df = train[train[location_type] == location].copy()
        location_df.drop([location_type, 'patient_id', 'DiagPeriodL90D'], axis=1, inplace=True)

        target_df = train[train[location_type] == location]['DiagPeriodL90D'].copy()
        
        # Store the filtered DataFrame in the dictionary with location as the key
        locations_dfs[location] = location_df
        target_dfs[location] = target_df

    return locations_dfs, target_dfs, train, test



division_dfs, division_target_dfs, division_train, division_test = group_by_location(train, test, 'Division')
region_dfs, region_target_dfs, region_train, region_test = group_by_location(train, test, 'Region')

In [53]:
# def apply_models_by_location(location_type, locations_dfs, target_dfs, train, test):

#     unique_locations = train[location_type].unique()

#     # Example usage for each slice
#     for location in tqdm(unique_locations):  # tqdm wrapper to show progress bar
#         important_features = apply_lasso(locations_dfs[location], target_dfs[location], alpha=0.1)
#         best_catboost_model, best_params, best_score = apply_catboost(locations_dfs[location][important_features], target_dfs[location], cv=2)
#         print("Best Parameters for", location, ":", best_params)
#         print("Best ROC for", location, ":", best_score)

#     return best_catboost_model

In [54]:
def apply_models_by_location(location_type, locations_dfs, target_dfs, train, test):

    unique_locations = train[location_type].unique()
    models_by_location = []  # List to store models for each location

    # Example usage for each slice
    for location in tqdm(unique_locations):  # tqdm wrapper to show progress bar
        important_features = apply_lasso(locations_dfs[location], target_dfs[location], alpha=0.1)
        best_catboost_model, best_params, best_score = apply_catboost(locations_dfs[location][important_features], target_dfs[location], cv=2)
        print("Best Parameters for", location, ":", best_params)
        print("Best ROC for", location, ":", best_score)
        models_by_location.append(best_catboost_model)  # Store the model for this location

    return models_by_location


In [56]:
def predict_by_location(test, models_by_location, location_type):
    predictions = {}  # Dictionary to store predictions for each location

    # Iterate over each unique location in the test data
    unique_locations = test[location_type].unique()
    for location in unique_locations:
        # Retrieve the model for the current location
        model_index = np.where(unique_locations == location)[0][0]
        model = models_by_location[model_index]

        # Filter test data for the current location
        location_test_data = test[test[location_type] == location].copy()
        location_test_data.drop([location_type, 'patient_id', 'DiagPeriodL90D'], axis=1, inplace=True)

        # Predict DiagPeriodL90D for the current location using the model
        location_predictions = model.predict(location_test_data)

        # Store the predictions for the current location
        predictions[location] = location_predictions

    return predictions


In [57]:
division_test

,patient_id,patient_race,payer_type,patient_zip3,patient_age,breast_cancer_diagnosis_code,metastatic_cancer_diagnosis_code,Division,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02
0,573710,0.659142,0.610440,467,54,0.797035,0.669078,0.623678,5441.435484,85.620968,40.880328,12.732258,14.088710,10.659677,11.625806,11.208065,15.619355,12.322581,8.409677,3.343548,49.154839,50.845161,55.175806,13.982258,24.266129,6.583871,3.073226,52.980645,66187.22807,1.611290,1.277419,2.645161,3.853226,3.172581,13.275806,12.633871,21.485484,16.717742,15.238710,8.070968,23.309677,33553.43333,84.112903,2064.741935,152749.5370,825.122449,23.895455,12.429032,40.667742,28.959677,11.895161,6.046774,17.941935,35.591379,63.303226,3.406557,10.655357,5.551786,94.793548,0.364516,0.303226,0.119355,0.009677,0.770968,3.630645,3.564516,13.996774,7.985484,0.969355,24.955357,10.838710,8.080645,38.724876,7.947165,11.157161
1,593679,0.605515,0.599005,337,52,0.776869,0.643646,0.605999,19613.820510,1555.107692,49.107692,8.069231,8.587179,10.684615,11.302564,10.971795,15.823077,15.902564,11.828205,6.815385,49.658974,50.341026,44.800000,17.779487,29.102564,8.310256,2.917105,46.665789,64711.71053,3.873684,2.044737,3.807895,4.239474,4.242105,9.347368,13.018421,17.373684,12.889474,14.442105,14.702632,29.144737,34678.61538,68.673684,8502.230769,265860.6053,1343.394737,34.957895,8.379487,26.558974,30.200000,22.100000,12.764103,34.864103,43.250000,57.035897,5.002632,11.564103,0.005128,78.217949,10.889744,3.453846,0.187179,0.076923,1.841026,5.328205,10.261538,16.020513,13.602564,2.836842,23.952632,10.579487,9.302564,36.918257,7.838973,13.599985
2,184532,0.624632,0.610440,917,61,0.788858,0.669078,0.643909,43030.500000,2048.578261,38.852174,11.306522,12.897826,14.121739,13.532609,13.160870,13.378261,11.473913,6.380435,3.736957,49.052174,50.947826,48.504348,10.117391,36.408696,4.969565,3.674783,59.219565,86330.39130,2.226087,1.528261,2.897826,2.747826,3.173913,6.647826,9.617391,15.965217,13.589130,19.752174,21.847826,41.600000,34317.82609,61.397826,12609.260870,572606.5000,1778.000000,34.595652,17.491304,22.656522,29.263043,20.200000,10.404348,30.604348,46.208696,63.154348,6.197826,15.708696,0.015217,38.708696,3.963043,25.565217,1.193478,0.269565,18.858696,11.426087,47.726087,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231
3,447383,0.624632,0.642286,917,64,0.776869,0.552356,0.643909,43030.500000,2048.578261,38.852174,11.306522,12.897826,14.121739,13.532609,13.160870,13.378261,11.473913,6.380435,3.736957,49.052174,50.947826,48.504348,10.117391,36.408696,4.969565,3.674783,59.219565,86330.39130,2.226087,1.528261,2.897826,2.747826,3.173913,6.647826,9.617391,15.965217,13.589130,19.752174,21.847826,41.600000,34317.82609,61.397826,12609.260870,572606.5000,1778.000000,34.595652,17.491304,22.656522,29.263043,20.200000,10.404348,30.604348,46.208696,63.154348,6.197826,15.708696,0.015217,38.708696,3.963043,25.565217,1.193478,0.269565,18.858696,11.426087,47.726087,9.895652,10.515217,12.745652,32

In [55]:
division_models = apply_models_by_location('Division', division_dfs, division_target_dfs, division_train, division_test)


 12%|█▎        | 1/8 [00:16<01:56, 16.66s/it]

Best Parameters for 0.6439094502335609 : {'depth': 8, 'iterations': 100, 'learning_rate': 0.1}
Best ROC for 0.6439094502335609 : 0.7799689543767165


 25%|██▌       | 2/8 [00:31<01:34, 15.83s/it]

Best Parameters for 0.6098748261474269 : {'depth': 4, 'iterations': 200, 'learning_rate': 0.01}
Best ROC for 0.6098748261474269 : 0.7619011984241054


 38%|███▊      | 3/8 [00:47<01:18, 15.70s/it]

...

Best Parameters for 0.5786290322580645 : {'depth': 4, 'iterations': 100, 'learning_rate': 0.01}
Best ROC for 0.5786290322580645 : 0.8006351287601288


 88%|████████▊ | 7/8 [02:09<00:21, 21.08s/it]

Best Parameters for 0.6059989832231825 : {'depth': 6, 'iterations': 100, 'learning_rate': 0.01}
Best ROC for 0.6059989832231825 : 0.7610712096556047


100%|██████████| 8/8 [02:34<00:00, 19.31s/it]

Best Parameters for 0.6662087912087912 : {'depth': 4, 'iterations': 300, 'learning_rate': 0.01}
Best ROC for 0.6662087912087912 : 0.8319294047167038


In [ ]:
predict_by_location(division_test, division_models, 'Division')

In [28]:
# region_models = apply_models_by_location('Region', region_dfs, region_target_dfs, region_train, region_test)


 25%|██▌       | 1/4 [00:29<01:27, 29.24s/it]

Best Parameters for 0.653120849933599 : {'depth': 8, 'iterations': 200, 'learning_rate': 0.01}
Best ROC for 0.653120849933599 : 0.7725209663103849


 50%|█████     | 2/4 [00:58<00:58, 29.46s/it]

Best Parameters for 0.6039477057164829 : {'depth': 4, 'iterations': 100, 'learning_rate': 0.01}
Best ROC for 0.6039477057164829 : 0.7624358144016703


 75%|███████▌  | 3/4 [01:23<00:27, 27.20s/it]

Best Parameters for 0.5903614457831325 : {'depth': 4, 'iterations': 100, 'learning_rate': 0.01}
Best ROC for 0.5903614457831325 : 0.7867999997713988


100%|██████████| 4/4 [01:50<00:00, 27.57s/it]

Best Parameters for 0.632139928942334 : {'depth': 4, 'iterations': 100, 'learning_rate': 0.05}
Best ROC for 0.632139928942334 : 0.7906790098910325


In [30]:
# Predict on the test data grouped by Division
division_predictions = predict_by_location('Division', division_test, division_models)

 11%|█         | 8/75 [00:00<00:01, 52.02it/s]


In [34]:
division_predictions

{'patient_id': 1,
 'patient_race': 0,
 'payer_type': 1,
 'patient_zip3': 1,
 'patient_age': 1,
 'breast_cancer_diagnosis_code': 1,
 'metastatic_cancer_diagnosis_code': 0,
 'Division': 0}

In [33]:
division_test

,patient_id,patient_race,payer_type,patient_zip3,patient_age,breast_cancer_diagnosis_code,metastatic_cancer_diagnosis_code,Division,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02
0,573710,0.659142,0.610440,467,54,0.797035,0.669078,0.623678,5441.435484,85.620968,40.880328,12.732258,14.088710,10.659677,11.625806,11.208065,15.619355,12.322581,8.409677,3.343548,49.154839,50.845161,55.175806,13.982258,24.266129,6.583871,3.073226,52.980645,66187.22807,1.611290,1.277419,2.645161,3.853226,3.172581,13.275806,12.633871,21.485484,16.717742,15.238710,8.070968,23.309677,33553.43333,84.112903,2064.741935,152749.5370,825.122449,23.895455,12.429032,40.667742,28.959677,11.895161,6.046774,17.941935,35.591379,63.303226,3.406557,10.655357,5.551786,94.793548,0.364516,0.303226,0.119355,0.009677,0.770968,3.630645,3.564516,13.996774,7.985484,0.969355,24.955357,10.838710,8.080645,38.724876,7.947165,11.157161
1,593679,0.605515,0.599005,337,52,0.776869,0.643646,0.605999,19613.820510,1555.107692,49.107692,8.069231,8.587179,10.684615,11.302564,10.971795,15.823077,15.902564,11.828205,6.815385,49.658974,50.341026,44.800000,17.779487,29.102564,8.310256,2.917105,46.665789,64711.71053,3.873684,2.044737,3.807895,4.239474,4.242105,9.347368,13.018421,17.373684,12.889474,14.442105,14.702632,29.144737,34678.61538,68.673684,8502.230769,265860.6053,1343.394737,34.957895,8.379487,26.558974,30.200000,22.100000,12.764103,34.864103,43.250000,57.035897,5.002632,11.564103,0.005128,78.217949,10.889744,3.453846,0.187179,0.076923,1.841026,5.328205,10.261538,16.020513,13.602564,2.836842,23.952632,10.579487,9.302564,36.918257,7.838973,13.599985
2,184532,0.624632,0.610440,917,61,0.788858,0.669078,0.643909,43030.500000,2048.578261,38.852174,11.306522,12.897826,14.121739,13.532609,13.160870,13.378261,11.473913,6.380435,3.736957,49.052174,50.947826,48.504348,10.117391,36.408696,4.969565,3.674783,59.219565,86330.39130,2.226087,1.528261,2.897826,2.747826,3.173913,6.647826,9.617391,15.965217,13.589130,19.752174,21.847826,41.600000,34317.82609,61.397826,12609.260870,572606.5000,1778.000000,34.595652,17.491304,22.656522,29.263043,20.200000,10.404348,30.604348,46.208696,63.154348,6.197826,15.708696,0.015217,38.708696,3.963043,25.565217,1.193478,0.269565,18.858696,11.426087,47.726087,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231
3,447383,0.624632,0.642286,917,64,0.776869,0.552356,0.643909,43030.500000,2048.578261,38.852174,11.306522,12.897826,14.121739,13.532609,13.160870,13.378261,11.473913,6.380435,3.736957,49.052174,50.947826,48.504348,10.117391,36.408696,4.969565,3.674783,59.219565,86330.39130,2.226087,1.528261,2.897826,2.747826,3.173913,6.647826,9.617391,15.965217,13.589130,19.752174,21.847826,41.600000,34317.82609,61.397826,12609.260870,572606.5000,1778.000000,34.595652,17.491304,22.656522,29.263043,20.200000,10.404348,30.604348,46.208696,63.154348,6.197826,15.708696,0.015217,38.708696,3.963043,25.565217,1.193478,0.269565,18.858696,11.426087,47.726087,9.895652,10.515217,12.745652,32

In [32]:
division_models

In [31]:
division_predictions

{'patient_id': 1,
 'patient_race': 0,
 'payer_type': 1,
 'patient_zip3': 1,
 'patient_age': 1,
 'breast_cancer_diagnosis_code': 1,
 'metastatic_cancer_diagnosis_code': 0,
 'Division': 0}

In [ ]:







# Predict on the test data grouped by Region
region_predictions = predict_by_location('Region', region_test, region_models)



In [ ]:
def predict_by_location(location_type, location_dfs, best_catboost_models):
    predicted_results = {}  # Dictionary to store predicted results for each location
    unique_locations = location_dfs.keys()

    for location in tqdm(unique_locations):  # tqdm wrapper to show progress bar
        model = best_catboost_models[location]
        data = location_dfs[location]
        # Assuming 'data' contains the features needed for prediction
        # Use the trained model to predict the target variable
        predictions = model.predict(data)
        # Store the predictions for this location
        predicted_results[location] = predictions

    return predicted_results


In [ ]:
# Predict on the test data grouped by Division
division_predictions = predict_by_location('Division', division_test, division_models)

  0%|          | 0/75 [00:00<?, ?it/s]


TypeError: 'CatBoostClassifier' object is not subscriptable

In [ ]:
# Predict on the test data grouped by Region
region_predictions = predict_by_location('Region', region_test, region_models)


In [ ]:
# state_models = apply_models_by_location('patient_state', state_dfs, state_target_dfs, state_train, state_test)